In [0]:
#This Code predict Label for SVF Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailsRoslyn_20102019_Single.csv')
data = data.sample(frac=1)


y = list(data['Assignee']) 
x = list(data['Title_Description']) 

le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

#80% / 20% train / test split:

train_size = int(len(x) * .8)

x_train = np.asarray(x_enc[:train_size])
y_train = np.asarray(y_enc[:train_size])

x_test = np.asarray(x_enc[train_size:])
y_test = np.asarray(y_enc[train_size:])

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [29]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [30]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=60, batch_size=32)
    model.save_weights('./Roslyn_TD_Assignee_Predict_Single.h5')

Epoch 1/60
715/715 [==============================] - 66s 93ms/step - loss: 3.5141 - acc: 0.2378
Epoch 2/60
715/715 [==============================] - 59s 83ms/step - loss: 2.7651 - acc: 0.3217
Epoch 3/60
715/715 [==============================] - 59s 83ms/step - loss: 2.5602 - acc: 0.3245
Epoch 4/60
715/715 [==============================] - 59s 83ms/step - loss: 2.4528 - acc: 0.3483
Epoch 5/60
715/715 [==============================] - 59s 83ms/step - loss: 2.3714 - acc: 0.3497
Epoch 6/60
715/715 [==============================] - 59s 83ms/step - loss: 2.3070 - acc: 0.3860
Epoch 7/60
715/715 [==============================] - 59s 83ms/step - loss: 2.2469 - acc: 0.3986
Epoch 8/60
715/715 [==============================] - 60s 84ms/step - loss: 2.1991 - acc: 0.4028
Epoch 9/60
715/715 [==============================] - 60s 84ms/step - loss: 2.1448 - acc: 0.4182
Epoch 10/60
715/715 [==============================] - 60s 84ms/step - loss: 2.0979 - acc: 0.4294
Epoch 11/60
715/715 [========

In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./Roslyn_TD_Assignee_Predict_Single.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [32]:
y_test = np.asarray(y_enc[train_size:])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))



                     precision    recall  f1-score   support

            333fred       0.25      0.25      0.25         4
          AlekseyTs       0.00      0.00      0.00         1
           AmadeusW       0.00      0.00      0.00         1
     CyrusNajmabadi       0.00      0.00      0.00         1
          JoeRobich       0.25      0.12      0.17         8
        RikkiGibson       0.50      0.09      0.15        11
             agocke       0.50      0.33      0.40         3
        allisonchou       0.00      0.00      0.00         2
             chborl       0.00      0.00      0.00         1
           chsienki       0.00      0.00      0.00         0
              cston       0.00      0.00      0.00         0
           dibarbet       0.00      0.00      0.00         3
         dotnet-bot       0.97      1.00      0.98        30
dotnet-maestro[bot]       1.00      1.00      1.00         1
             gafter       0.20      0.32      0.25        22
              genlu    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# New Section